In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

In [2]:
!pip install transformers

In [3]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

model_dir = "/content/drive/My Drive/deberta-v3-finance-classification"
model_id = "nickmuchi/deberta-v3-base-finetuned-finance-text-classification"
model = AutoModelForSequenceClassification.from_pretrained(model_id)
model.save_pretrained(model_dir)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [6]:
import pandas as pd
import json
from google.colab import drive
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import os
from transformers import pipeline

drive.mount('/content/drive')

model_dir = "/content/drive/My Drive/deberta-v3-finance-classification"

model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Device set to use cuda:0


In [7]:
def normalize_label(label):
  return label.lower().strip()

def map_label(label):
  label_mapping = { "bullish": "positive", "neutral": "neutral", "bearish": "negative" }
  return label_mapping.get(label, label)

In [8]:
import pandas as pd

df_actualite = pd.read_csv(
    "allactualite.csv",
    sep=",",
    quotechar='"',
    encoding="utf-8",
    on_bad_lines="skip"
)
df_seance = pd.read_csv("allseance.csv", sep=",", quotechar='"', encoding="utf-8")

print(df_seance.head())
print(df_actualite.head())

  date_seance  prix_cloture_ajuste         volume  qte_echangee     code_isin  \
0  2018-12-31            17.237187    2121.020020            78  TN0003400058   
1  2018-12-31             1.877000   23547.750000          9415  TN0006590012   
2  2018-12-31             7.329000    3229.719971           422  TN0001000108   
3  2018-12-31             5.065000   60538.839844         10676  TN0006530018   
4  2018-12-31            11.383000  112094.242188          6010  TN0007570013   

  libelle_long  
0    AMEN BANK  
1        SIAME  
2     MONOPRIX  
3      SOTETEL  
4  EURO-CYCLES  
  date_publication     code_isin  \
0       2023-12-18  TN0007560014   
1       2021-10-18  TN0007560014   
2       2018-08-22  TN0007560014   
3       2018-07-18  TN0007560014   
4       2018-07-13  TN0007560014   

                                         description  
0  La Chambre civile de la Cour d''appel a décidé...  
1  La compagnie aérienne privée Syphax Airlines a...  
2  La compagnie aérienne Syph

In [9]:
import pandas as pd
import os

df_matched = pd.merge(df_actualite, df_seance, how='inner',
                      left_on=['date_publication', 'code_isin'],
                      right_on=['date_seance', 'code_isin'])

#les descriptions en chaînes
df_matched['description'] = df_matched['description'].astype(str)

batch_size = 400
output_file_actualite = "allActualiteLabellise.csv"
output_file_seance = "AllSeanceLabellise.csv"

# Charger les résultats partiels si existants
if os.path.exists(output_file_actualite):
    df_actualite = pd.read_csv(output_file_actualite)
    df_seance = pd.read_csv(output_file_seance)
    processed_descriptions = set(df_actualite['description'].dropna())
else:
    df_actualite['score'] = 'NaN'
    processed_descriptions = set()

# application du modèle DeBERta
def classify_and_save(df_batch, start_index):
    results = []
    for i, row in df_batch.iterrows():
        description = row['description']
        if description in processed_descriptions:
            continue
        try:
            if description.strip():
                score = map_label(normalize_label(classifier(description)[0]['label']))
            else:
                score = 'NaN'
        except Exception as e:
            score = 'Error'
            print(f"Erreur avec la description {description}: {e}")
        results.append((row['date_seance'], row['code_isin'], description, score))

        processed_descriptions.add(description)

    for _, _, description, score in results:
        df_actualite.loc[df_actualite['description'] == description, 'score'] = score

    for date_seance, code_isin, _, score in results:
        df_seance.loc[(df_seance['date_seance'] == date_seance) &
                      (df_seance['code_isin'] == code_isin), 'score'] = score

    # Sauvegarder les résultats
    df_actualite.to_csv(output_file_actualite, index=False, encoding="utf-8")
    df_seance.to_csv(output_file_seance, index=False, encoding="utf-8")

    return len(results)

for start_index in range(0, len(df_matched), batch_size):
    batch = df_matched.iloc[start_index:start_index + batch_size]
    print(f"Traitement du lot {start_index} à {start_index + batch_size}")
    classify_and_save(batch, start_index)
    print(f"Lot {start_index} terminé. Résultats sauvegardés.")


Traitement du lot 0 à 400


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Lot 0 terminé. Résultats sauvegardés.
Traitement du lot 400 à 800
Lot 400 terminé. Résultats sauvegardés.
Traitement du lot 800 à 1200
Lot 800 terminé. Résultats sauvegardés.
Traitement du lot 1200 à 1600
Lot 1200 terminé. Résultats sauvegardés.
Traitement du lot 1600 à 2000
Lot 1600 terminé. Résultats sauvegardés.
Traitement du lot 2000 à 2400
Lot 2000 terminé. Résultats sauvegardés.
Traitement du lot 2400 à 2800
Lot 2400 terminé. Résultats sauvegardés.
Traitement du lot 2800 à 3200
Lot 2800 terminé. Résultats sauvegardés.
Traitement du lot 3200 à 3600
Lot 3200 terminé. Résultats sauvegardés.
Traitement du lot 3600 à 4000
Lot 3600 terminé. Résultats sauvegardés.
Traitement du lot 4000 à 4400
Lot 4000 terminé. Résultats sauvegardés.


In [10]:
import pandas as pd

df_actualite = pd.read_csv("allActualiteLabellise.csv")
df_seance = pd.read_csv("AllSeanceLabellise.csv")

valid_labels = ["neutral", "positive", "negative"]
# Filtrer les lignes
df_actualite_labellise = df_actualite[df_actualite['score'].isin(valid_labels)]
df_seance_labellise = df_seance[df_seance['score'].isin(valid_labels)]

df_actualite_labellise.to_csv("df_actualite_labellise.csv", index=False, encoding="utf-8")
df_seance_labellise.to_csv("df_seance_labellise.csv", index=False, encoding="utf-8")

print(f"Nombre de lignes labellisées - Actualité : {len(df_actualite_labellise)}")
print(f"Nombre de lignes labellisées - Séance : {len(df_seance_labellise)}")


Nombre de lignes labellisées - Actualité : 4243
Nombre de lignes labellisées - Séance : 4138


In [17]:
import pandas as pd

df = pd.read_csv('df_seance_labellise.csv')
df['sentiment_score'] = df['score'].map({
    'positive': 1,
    'negative': -1,
    'neutral': 0
})

df.to_csv('seance_labelliseSentiment.csv', index=False)


In [20]:
import pandas as pd

df = pd.read_csv('df_actualite_labellise.csv')
df['sentiment_score'] = df['score'].map({
    'positive': 1,
    'negative': -1,
    'neutral': 0
})

df.to_csv('DataActualiteLabellise.csv', index=False)


In [2]:
import pandas as pd

df = pd.read_csv('seance.csv')
df = df.sort_values(by=['code_isin', 'date_seance'])
df['trend'] = df.groupby('code_isin')['prix_cloture_ajuste'].shift(-1)


df.to_csv('trend.csv', index=False)


In [5]:
import pandas as pd

df = pd.read_csv('trend.csv')
df = df.sort_values(by=['code_isin', 'date_seance'])

# Calculer la variation du prix_cloture_ajuste entre J et J-1
df['variation'] = df.groupby('code_isin')['prix_cloture_ajuste'].diff()

epsilon = 1e-6
df['tendance'] = df['variation'].apply(
    lambda x: 'constante' if abs(x) < epsilon else ('hausse' if x > 0 else 'baisse')
)


df['trend'] = df.groupby('code_isin')['prix_cloture_ajuste'].shift(-1)
df['tendance'] = df.groupby('code_isin')['tendance'].shift(-1)

df.drop(columns=['variation'], inplace=True)

df.to_csv('tendance.csv', index=False)
